使用工具

In [7]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = "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"

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1744015917, 'localtime': '2025-04-07 01:51'}, 'current': {'last_updated_epoch': 1744015500, 'last_updated': '2025-04-07 01:45', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 6.0, 'wind_kph': 9.7, 'wind_degree': 203, 'wind_dir': 'SSW', 'pressure_mb': 1023.0, 'pressure_in': 30.2, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 80, 'cloud': 75, 'feelslike_c': 12.0, 'feelslike_f': 53.6, 'windchill_c': 11.5, 'windchill_f': 52.7, 'heatindex_c': 12.5, 'heatindex_f': 54.6, 'dewpoint_c': 12.0, 'dewpoint_f': 53.6, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 9.8, 'gust_kph': 15.7}}", 'score'

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o", model_provider="openai")

In [11]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [12]:
model_with_tools = model.bind_tools(tools)

In [13]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [15]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco current weather'}, 'id': 'call_mn7iriSwpj5o63NlPQqp3jKY', 'type': 'tool_call'}]


In [16]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [17]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='ef84e149-d3e0-4307-86ab-f88794bd0efc'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 81, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BJceUa33l4pg9PhxRJszgu2WfMNFI', 'finish_reason': 'stop', 'logprobs': None}, id='run-11f5f68b-8a24-460e-a374-c5e8cdc2d1db-0', usage_metadata={'input_tokens': 81, 'output_tokens': 11, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [18]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='dc1f8708-49e6-47b7-8736-00de4a7e462f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rRasqor21MiSryGDPxndykec', 'function': {'arguments': '{"query":"San Francisco weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BJcg7rf7WZnC9Q2At26klwsf09wat', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7ecfb3ea-c42e-4f22-943e-32876f24dcd7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weat